In [1]:
#ante-MMPBSA.py -p 0.15_80_10_pH7.5_6m0j_trunc_final.top -r ACE2_dry_trunc.prmtop -l SARS-CoV-2-spike_dry_trunc.prmtop -m :1-339
# WARNING: -m is recepter residue number not atom number
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import conda_installer
from rdkit import Chem
from deepchem.feat.graph_features import atom_features as get_atom_features
import rdkit
import pickle
import matplotlib.pyplot as plt

2023-11-02 19:51:26.252442: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-02 19:51:26.469763: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-02 19:51:26.472339: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-02 19:51:27.470903: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
PDBs = pickle.load(open('PDBs_RDKit.pkl', 'rb'))
PDBs_BRD4 = pickle.load(open('PDBs_BRD4.pkl', 'rb'))
df = pd.read_csv('T_data.csv')

In [3]:
%cd ../../PGGCN/

/home/ali/GBNN/PGGCN


In [4]:
info = []
for pdb in list(PDBs.keys()):
    info.append(df[df['Id'] == pdb][['TS_comp', 'TS_host', 'TS_ligand']].to_numpy()[0])

In [5]:
from models.dcFeaturizer import atom_features as get_atom_features
def featurize(molecule, info):
    
    atom_features = []
    for atom in molecule.GetAtoms():
        new_feature = get_atom_features(atom).tolist()
        position = molecule.GetConformer().GetAtomPosition(atom.GetIdx())
        new_feature += [atom.GetMass(), atom.GetAtomicNum()]
        new_feature += [position.x, position.y, position.z]
        for neighbor in atom.GetNeighbors()[:2]:
            neighbor_idx = neighbor.GetIdx()
            new_feature += [neighbor_idx]
        for i in range(2 - len(atom.GetNeighbors())):
            new_feature += [-1]
        atom_features.append(np.concatenate([new_feature, info], 0))
    return np.array(atom_features)

In [6]:
X = []
y = []
for i, pdb in enumerate(list(PDBs.keys())):
    X.append(featurize(PDBs[pdb], info[i]))
    y.append(df[df['Id'] == pdb]['exp'].to_numpy()[0])

In [7]:
X[0].shape

(100, 41)

In [7]:
X_brd4 = []
y_brd4 = []
info_brd4 = []
for i,pdb in enumerate(list(PDBs_BRD4.keys())):
    info_brd4.append(df[df['Id'] == pdb][['TS_comp', 'TS_host', 'TS_ligand']].to_numpy()[0])
for i,pdb in enumerate(list(PDBs_BRD4.keys())):
    X_brd4.append(featurize(PDBs_BRD4[pdb], info_brd4[i]))
    y_brd4.append(df[df['Id'] == pdb]['exp'].to_numpy()[0])

In [8]:
X_train, y_train = X, y
info_train = info

In [9]:
X_c = X.copy()
y_c = y.copy()

In [10]:
from sklearn.utils import shuffle
X, y = shuffle(X, y)

In [11]:
import models.layers_update_mobley as layers
import importlib
# import tensorflow_addons as tfa
importlib.reload(layers)


class PGGCNModel(tf.keras.Model):
    def __init__(self, num_atom_features = 36, r_out_channel = 20, c_out_channel = 128):
        super().__init__()
        self.ruleGraphConvLayer = layers.RuleGraphConvLayer(r_out_channel, num_atom_features, 0)
        self.ruleGraphConvLayer.combination_rules = []
        self.conv = layers.ConvLayer(c_out_channel)
        self.dense1 = tf.keras.layers.Dense(64, activation='relu', name='dense1')
#         self.dense2 = tf.keras.layers.Dense(32, activation='sigmoid')
#         self.dense3 = tf.keras.layers.Dense(100, activation='relu')
#         self.dense4 = tf.keras.layers.Dense(80, activation='relu')
        self.dense5 = tf.keras.layers.Dense(16, name='relu')
        self.dense6 = tf.keras.layers.Dense(1, name='dense6')
#         self.dense7 = tf.keras.layers.Dense(1, name='dense7',
#                  kernel_initializer=tf.keras.initializers.Constant([-.2, -1, 1, 1]),
#                  bias_initializer=tf.keras.initializers.Zeros())
        self.all_layer_1_weights = []
        
    def addRule(self, rule, start_index, end_index = None):
        self.ruleGraphConvLayer.addRule(rule, start_index, end_index)
    
    def set_input_shapes(self, i_s):
        self.i_s = i_s

    def call(self, inputs):
        physics_info = inputs[:,0,38:]
        x_a = []
        for i in range(len(self.i_s)):
            x_a.append(inputs[i][:self.i_s[i], :38])
#         agg = []
#         for i in range(len(x_a)):
#             agg.append([x_a[i], self.a_l[i]])
        x = self.ruleGraphConvLayer(x_a)
        self.all_layer_1_weights.append(self.ruleGraphConvLayer.w_s)
        x = self.conv(x)
        x = self.dense1(x)
#         x = self.dense2(x)
#         x = self.dense3(x)
#         x = self.dense4(x)
        x = self.dense5(x)
        model_var = self.dense6(x)
#         merged = tf.concat([model_var, physics_info], axis=1)
#         out = self.dense7(merged)
        return model_var

m = PGGCNModel()
m.addRule("sum", 0, 31)
m.addRule("multiply", 31, 33)
m.addRule("distance", 33, 36)

import keras.backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred[0] - y_true))) + K.abs(1 / K.mean(.2 + y_pred[1]))
def pure_rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))
    

# optimizers = [
#     tf.keras.optimizers.Adam(learning_rate=0.1),
#     tf.keras.optimizers.Adam(learning_rate=0.005)
# ]
# optimizers_and_layers = [(optimizers[0], m.layers[:len(m.layers) - 1]), 
#                          (optimizers[1], m.layers[len(m.layers) - 1])]    
# optimizer = tfa.optimizers.MultiOptimizer(optimizers_and_layers)

# opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
m.compile(loss=pure_rmse, optimizer='adam')#opt)
X_train, y_train = X, y
info_train = info

input_shapes = []
for i in range(len(X_train)):
    input_shapes.append(np.array(X_train[i]).shape[0])
m.set_input_shapes(input_shapes)
for i in range(len(X_train)):
    if X_train[i].shape[0] < 2000:
        new_list = np.zeros([2000 - X_train[i].shape[0], 41])
        X_train[i] = np.concatenate([X_train[i], new_list], 0)
X_train = np.array(X_train)
y_train = np.array(y_train)
# callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
hist = m.fit(X_train, y_train, epochs = 40, batch_size=81)

2023-08-02 20:40:57.037924: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/40


2023-08-02 20:40:57.279448: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 26568000 exceeds 10% of free system memory.
2023-08-02 20:42:22.474766: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 26568000 exceeds 10% of free system memory.
2023-08-02 20:42:22.475084: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 26568000 exceeds 10% of free system memory.
2023-08-02 20:42:22.477393: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 26568000 exceeds 10% of free system memory.
2023-08-02 20:42:22.477508: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 26568000 exceeds 10% of free system memory.


1/1 [==============================] - 86s 86s/step - loss: 2.1296
Epoch 2/40
1/1 [==============================] - 0s 339ms/step - loss: 2.1279
Epoch 3/40
1/1 [==============================] - 1s 687ms/step - loss: 2.1263
Epoch 4/40
1/1 [==============================] - 1s 728ms/step - loss: 2.1246
Epoch 5/40
1/1 [==============================] - 1s 699ms/step - loss: 2.1230
Epoch 6/40
1/1 [==============================] - 1s 744ms/step - loss: 2.1214
Epoch 7/40
1/1 [==============================] - 1s 759ms/step - loss: 2.1197
Epoch 8/40
1/1 [==============================] - 1s 728ms/step - loss: 2.1181
Epoch 9/40
1/1 [==============================] - 1s 705ms/step - loss: 2.1165
Epoch 10/40
1/1 [==============================] - 1s 692ms/step - loss: 2.1149
Epoch 11/40
1/1 [==============================] - 1s 686ms/step - loss: 2.1133
Epoch 12/40
1/1 [==============================] - 1s 713ms/step - loss: 2.1117
Epoch 13/40
1/1 [==============================] - 1s 695ms/s

In [12]:
X_test = X_brd4
y_test = y_brd4
info_test = info_brd4
input_shapes = []
for i in range(len(X_test)):
    input_shapes.append(np.array(X_test[i]).shape[0])
m.set_input_shapes(input_shapes)
for i in range(len(X_test)):
    if X_test[i].shape[0] < 2000:
        new_list = np.zeros([2000 - X_test[i].shape[0], 41])
        X_test[i] = np.concatenate([X_test[i], new_list], 0)
X_test = np.array(X_test)
y_test = np.array(y_test)
m.evaluate(X_test, y_test)

1/1 [==============================] - 2s 2s/step - loss: 2.7530


2.7529828548431396

In [24]:
with open('PGCN_DD_single.pkl', 'wb') as f:
    pickle.dump([hist.history['loss'], 3.01], f)